In [4]:
import json
# Load the utilities JSON
def load_json(address):
    with open(address, 'r', encoding='utf-8') as f:
        return json.load(f)

In [16]:
train_msg = load_json(r"/home/hz271/Research/General Robotics Lab 通用机器人实验室/LLM_As_Agent/data/input/msg/train_msg.json")

In [17]:
print(train_msg["desc_user_0"])

This is a text-based small 2D world (world ID #{}), you mission is to explore it as much as you can, understand surrounding's object's properties and summarize useful experience. Observations are composed of: 
 1. World Map: A 2D matrix that records yourself and surrounding objects, where each object is encoded by this rule: If 5 is grey, then 5Wall is grey wall. 1 is green, 1Door+ will mean a green door but locked because + represents locked. Yourself is the position of you in the world map, with coordinate being (X, Y) and your facing direction (Right/East, Up/North, Left/West or Down/South). You cannot see objects behind you or blocked behind. The world map will be updated as you explore around. You will forget about positions that you haven't explored for a while. Places you have forgotten or have not viewed yet will all be marked as unseen (-).
 An example:
[[- 5Wall 5Wall 5Wall 5Wall 5Wall - - - - -] 
 [- 0Empty 0Empty 0Empty 1Ball 1Door+ - - - - -] 
 [- Yourself 0Empty 0Empty 1K

In [18]:
import tiktoken

In [19]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [25]:
a = """This is a text-based small 2D world (world ID #{}), you mission is to explore it as much as you can, understand surrounding's object's properties and summarize useful experience. Observations are composed of: 
 1. World Map: A 2D matrix that records yourself and surrounding objects, where each object is encoded by this rule: If 5 is grey, then 5Wall is grey wall. 1 is green, 1Door+ will mean a green door but locked because + represents locked. Yourself is the position of you in the world map, with coordinate being (X, Y) and your facing direction (Right/East, Up/North, Left/West or Down/South). You cannot see objects behind you or blocked behind. The world map will be updated as you explore around. You will forget about positions that you haven't explored for a while. Places you have forgotten or have not viewed yet will all be marked as unseen (-).
 An example:
[[- 5Wall 5Wall 5Wall 5Wall 5Wall - - - - -] 
 [- 0Empty 0Empty 0Empty 1Ball 1Door+ - - - - -] 
 [- Yourself 0Empty 0Empty 1Key 5Wall - - - - -] 
 [- 0Empty 0Empty 0Empty 0Empty 5Wall - - - - -] 
 [- 0Empty 0Empty 0Empty 0Empty 5Wall - - - - -] 
 [- 5Wall 5Wall 5Wall 5Wall 5Wall - - - - -]] 
 The numbers represents color, where black is 0, green is 1, blue is 2, purple is 3, yellow is 4 and grey is 5. Symbols represent status, nothing is open, + is locked, * is closed. For example 5Door* is a green closed door. Your position (X, Y) and facing direction. For example in this world map your position is (2,1) and direction is East/Right. Interpretation can be: a. You are in a room with walls that are grey (5Wall) on the top, bottom, and right side. b. Three spaces in front of you, to your left/west, is a green ball (1Ball). c. Four spaces in front of you, one space to your left, is a green locked (+) door (1Door+). 
 2. Inventory. You can only hold one item with you. For example you are holding a key, then you can't pick up anything unless you dropped off your key to an droppable object. 
 3. Past actions. You can remember your latest few past actions, try using diverse actions as they help to explore the world. An example:  Your past actions are forward, forward 
 Interpretation: You have tried forward two times.
4. Gained experience:
You can refer to your gained past experiences to guide you to explore the world. 
 All these examples are just for your reference, your real observation will be provided to you later with a similar format."""

In [26]:
len(encoding.encode(a))

657

In [22]:
len(encoding.encode("what"))

1

In [9]:
import os
os.curdir

'.'

In [10]:
import openai
openai.api_key = open("../utils/api/API_KEY", "r").read()

In [93]:
def choose_act(action):
    # Check if param is a list
    if isinstance(action, list):
        # Check if the list has more than 1 element
        if len(action) > 1:
            return action
        else:
            # If it's a list of length 1, extract the first element and try to convert it to int
            try:
                return int(action[0])
            except (ValueError, TypeError):
                raise ValueError("Single element in the list or the parameter itself could not be converted to an integer.")
    else:
        # If it's not a list, try to convert it to int
        try:
            return int(action)
        except (ValueError, TypeError):
            raise ValueError("The parameter could not be converted to an integer.")

In [94]:
msg = [{"role": "user", "content": "convert 1, 2 into a list of integers by calling function \"choose_act\" with list of integers as parameter"}]

In [101]:
msg = [{"role": "user", "content": "Choose an action or a mutiple step actions plan from the following to exlore the environment as much as you can: \n(0) left: turn your view to the left object\n(1) right: turn your view to the right object\n(2) toggle: toggle the object in front of you\n(3) forward: move yourself to the front object if it's empty or opened\n(4) pick up: pick up the front object if it can be picked up\n(5) drop off: drop off the current inventory if front is empty\n\nAnd explain why.\n\nLimit your reason to be less than 100 tokens.\n\nUse format like: (a). Action #1 (b). Action #2 etc."}]
msg.append({"role": "assistant", "content": "I choose to do move forward four times"})
msg.append({"role": "user", "content": "Convert above reason of choices into a list of integers by calling function \"choose_act\" with a list of integers as parameter"})

In [102]:

fuc_msg = "Function to convert the reason of choices into a list of integers"
fuc = [
    {
        "name": "choose_act",
        "description":fuc_msg,
        "parameters":{
            "type":"object",
            "properties":{
                "action":
                {
                    "type":"integer",
                    "description":"list of integers corresponding to the action",
                }  
            },
            "required":["action"]
        }
    }]

In [103]:
import json
rsp = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = msg,
    functions = fuc,
    function_call = "auto",
    temperature = 0.1
)
rsp_msg = rsp["choices"][0]["message"]
if rsp_msg.get("function_call"):
    fuc_l = {
        "choose_act": choose_act,
    }
    fuc_n = rsp_msg["function_call"]["name"]
    if fuc_n == "choose_act":
        fuc_c = fuc_l[fuc_n]
        fuc_args = json.loads(rsp_msg["function_call"]["arguments"])
        act = fuc_c(
            action = fuc_args.get("action")
        )
        

In [104]:
type(act)

list

In [105]:
act

[3, 3, 3, 3]

In [106]:
rsp["choices"]

[<OpenAIObject at 0x7f504f2735b0> JSON: {
   "index": 0,
   "message": {
     "role": "assistant",
     "content": null,
     "function_call": {
       "name": "choose_act",
       "arguments": "{\n  \"action\": [3, 3, 3, 3]\n}"
     }
   },
   "finish_reason": "function_call"
 }]

In [107]:
print(rsp_msg)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "choose_act",
    "arguments": "{\n  \"action\": [3, 3, 3, 3]\n}"
  }
}
